In [4]:
import threading


class State:

    def __init__(self, name, automaton) -> None:
        self.name = name
        self.automaton = automaton

    def on_enter(self):
        """
        Called when the state is entered.
        """
        pass
    
    def on_event(self, event):
        """
        Handle events that are passed to the state. 
        Returns the next state or itself if no transition occurs.
        """
        pass

    def __str__(self) -> str:
        return self.name
    

class TimeoutState(State):

    def __init__(self, name, automaton, timeout=None, timeout_event=None) -> None:
        super().__init__(name, automaton)
        self.timeout = timeout
        self.timeout_event = timeout_event
        self.timer = None

    def on_enter(self):
        if self.timeout and self.timeout_event:
            self.start_timer()

    def start_timer(self):
        """
        Start a timer to automatically trigger an event after a timeout.
        """
        self.timer = threading.Timer(self.timeout, self.trigger_timeout_event)
        self.timer.start()

    def trigger_timeout_event(self):
        """
        Trigger the timeout event if no other event occurs.
        """
        self.automaton.on_event(self.timeout_event)

    def on_event(self, event):
        
        # Cancel the timer to avoid firing the timout event
        self.cancel_timer()

        # Handle the event received
        # ...

    def cancel_timer(self):
        """
        Cancel the timeout timer if an event is received.
        """
        if self.timer:
            self.timer.cancel()
            self.timer = None

In [16]:
class SteadyState(State):

    def __init__(self, automaton) -> None:
        super().__init__('steady_state', automaton)

    def on_enter(self):
        super().on_enter()
        print("[Automaton] Entering Steady State")

    def on_event(self, event):
        super().on_event(event)
        if event == 'hand_touched':
            self.automaton.change_state('moving_state')

class MovingState(State):

    def __init__(self, automaton) -> None:
        super().__init__('moving_state', automaton)

    def on_enter(self):
        super().on_enter()
        print("[Automaton] Entering Moving State")

    def on_event(self, event):
        super().on_event(event)
        if event == 'hand_released':
            self.automaton.change_state('ask_state')
        elif event == 'goal_reached':
            self.automaton.change_state('goal_state')

class GoalState(State):

    def __init__(self, automaton) -> None:
        super().__init__('goal_state', automaton)

    def on_enter(self):
        super().on_enter()
        print("[Automaton] Entering Goal State")
    
    def on_event(self, event):
        super().on_event(event)
        # No event handling, program terminates

class AskState(TimeoutState):

    def __init__(self, automaton) -> None:
        super().__init__('ask_state', automaton, timeout=20, timeout_event='steady_state')
    
    def on_enter(self):
        super().on_enter()
        print("[Automaton] Entering Ask State")
    
    def on_event(self, event):
        super().on_event(event)
        if event == 'response_yes':
            self.automaton.change_state('steady_state')
        elif event == 'response_no':
            self.automaton.change_state('ask_2_state')
        elif event == 'hand_touched':
            self.automaton.change_state['moving_state']

class Ask2State(TimeoutState):

    def __init__(self, automaton) -> None:
        super().__init__('ask_2_state', automaton, timeout=20, timeout_event='steady_state')

    def on_enter(self):
        super().on_enter()
        print("[Automaton] Entering Ask2 State")

    def on_event(self, event):
        super().on_event(event)
        if event == 'hand_touched':
            self.automaton.change_state['moving_state']

class FiniteStateAutomaton:
    def __init__(self):
        self.states = {}
        self.current_state = None

    def add_state(self, state):
        self.states[state.name] = state

    def set_initial_state(self, state_name):
        if state_name not in self.states:
            raise ValueError(f"State '{state_name}' does not exist.")            
        self.current_state = self.states[state_name]

    def change_state(self, state_name):
        if state_name in self.states:
            self.current_state = self.states[state_name]
            self.current_state.on_enter()
        else:
            raise ValueError(f"State '{state_name}' does not exist.")

    def on_event(self, event):
        if self.current_state is None:
            raise ValueError(f"Automaton has not been initialized yet.")
        self.current_state.on_event(event)

In [17]:
automaton = FiniteStateAutomaton()

steady_state = SteadyState(automaton)
moving_state = MovingState(automaton)
goal_state = GoalState(automaton)
ask_state = AskState(automaton, timeout=10)
ask2_state = Ask2State(automaton, timeout=10)

automaton.add_state(steady_state)
automaton.add_state(moving_state)
automaton.add_state(goal_state)
automaton.add_state(ask_state)
automaton.add_state(ask2_state)

automaton.set_initial_state('steady_state')

In [22]:
import time
import random

automaton.on_event('hand_touched')

# Now we simulate the user releasing the hand of the robot while it is moving

print('[User] releasing hand')
automaton.on_event('hand_released')

choice = random.randint(0, 2)  # 0 to fire hand_touched, 1 to fire 'response_no'

time_to_sleep = random.randint(5, 15)
time.sleep(time_to_sleep)
print(f'[User] waiting for {time_to_sleep} seconds')

choice = 'yes' if random.random() > 0.5 else 'no'
print(f'[User] responding: {choice}')
automaton.on_event(f'response_{choice}')

[User] releasing hand
[Automaton] Entering Ask State
[User] waiting for 15 seconds
[User] responding: yes
[Automaton] Entering Steady State
